<a href="https://colab.research.google.com/github/sadra-barikbin/novel-solutions-for-sentiment-analysis/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lxml==4.3.2 tqdm==4.56.0 torch==1.3.1

In [ ]:
! pip install matplotlib==3.1.2 gensim==3.8.1 stanza==1.1.1 nltk==3.4.5 torchtext==0.3.1

In [ ]:
import stanza
stanza.download('en')
stanza.download('es')
stanza.download('eu')
stanza.download('ca')
stanza.download('nb')

2022-04-30 12:29:50 INFO: Downloading default packages for language: en (English)...
2022-04-30 12:31:16 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-30 12:31:16 INFO: Downloading default packages for language: es (Spanish)...
2022-04-30 12:33:12 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-30 12:33:12 INFO: Downloading default packages for language: eu (Basque)...
2022-04-30 12:33:56 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-30 12:33:56 INFO: Downloading default packages for language: ca (Catalan)...
2022-04-30 12:34:41 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-30 12:34:41 INFO: Downloading default packages for language: nb (Norwegian_Bokmaal)...
2022-04-30 12:36:45 INFO: Finished downloading models and saved to /root/stanza_resources.


Our baselines are the ones introduced in the task github repo. First one takes a dependency graph parsing approach and is originally based on [(Barnes et al. 2021)](https://arxiv.org/abs/2105.14504). Second is a simple sequence labelling LSTM.

In [ ]:
! git clone https://github.com/sadra-barikbin/semeval22_structured_sentiment.git

Cloning into 'semeval22_structured_sentiment'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 993 (delta 59), reused 69 (delta 45), pack-reused 894
Receiving objects: 100% (993/993), 16.08 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (488/488), done.


In [ ]:
! cd semeval22_structured_sentiment/data/mpqa && bash process_mpqa.sh
! cd semeval22_structured_sentiment/data/darmstadt_unis && bash process_darmstadt.sh

In [ ]:
%%bash
cd semeval22_structured_sentiment/baselines/graph_parser
if [ -d embeddings ]; then
    echo "Using downloaded word embeddings"
else
    mkdir embeddings
    cd embeddings
    wget http://vectors.nlpl.eu/repository/20/58.zip
    wget http://vectors.nlpl.eu/repository/20/32.zip
    wget http://vectors.nlpl.eu/repository/20/34.zip
    wget http://vectors.nlpl.eu/repository/20/18.zip
    wget http://vectors.nlpl.eu/repository/20/68.zip
cd ..
fi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Baseline 1 - Dependency Graph Parsing

In [ ]:
! cd semeval22_structured_sentiment/baselines/graph_parser && bash get_baseline.sh

2022-04-30 12:38:32 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2022-04-30 12:38:32 INFO: Use device: gpu
2022-04-30 12:38:32 INFO: Loading: tokenize
2022-04-30 12:38:32 INFO: Loading: pos
2022-04-30 12:38:35 INFO: Loading: lemma
2022-04-30 12:38:35 INFO: Loading: depparse
2022-04-30 12:38:36 INFO: Done loading processors!
Dataset: ../../data/darmstadt_unis
Setup: head_final
2022-04-30 12:40:08 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2022-04-30 12:40:08 INFO: Use device: gpu
2022-04-30 12:40:08 INFO: Loading: tokenize
2022-04-30 12:40:08 INFO: Loading: pos
2022-04-30 12:40:11 INFO: Loading: lemma
2022-04-30 12:40:11 INFO: Loading: depparse
2022-04-30 12:40:12 INFO: Done loadi

In [ ]:
%%bash
cd semeval22_structured_sentiment/baselines/graph_parser/
for DATASET in darmstadt_unis mpqa multibooked_ca multibooked_eu norec opener_es opener_en; do
    python3 convert_to_conllu.py --json_dir ../../data/"$DATASET" --out_dir sentiment_graphs/"$DATASET" --setup head_final
done;

In [ ]:
%%bash
cd semeval22_structured_sentiment/baselines/graph_parser
for ds in opener_en opener_es darmstadt_unis multibooked_ca multibooked_eu; do
  if [ $ds == norec ]; then
      embed=embeddings/58.zip
  elif [ $ds == multibooked_eu ]; then
      embed=embeddings/32.zip
  elif [ $ds == multibooked_ca ]; then
      embed=embeddings/34.zip
  elif [ $ds == mpqa ]; then
      embed=embeddings/18.zip
  elif [ $ds == darmstadt_unis ]; then
      embed=embeddings/18.zip
  elif [ $ds == opener_en ]; then
      embed=embeddings/18.zip
  elif [ $ds == opener_es ]; then
      embed=embeddings/68.zip
  else
      echo "NO EMBEDDINGS SUPPLIED FOR THIS DATASET"
      exit
  fi
  bash ./inference.sh sentiment_graphs/$ds/head_final/test.conllu experiments/$ds/head_final $embed
done;

In [ ]:
%%bash
cd semeval22_structured_sentiment/evaluation
for ds in mpqa norec; do
  echo $ds:$(python3 evaluate_single_dataset.py ../data/$ds/test.json ../baselines/graph_parser/experiments/$ds/head_final/test.conllu.json)
done;

mpqa:Sentiment Tuple F1: 0.159
norec:Sentiment Tuple F1: 0.282


In [ ]:
%%bash
cd semeval22_structured_sentiment/evaluation
for ds in opener_en opener_es darmstadt_unis multibooked_ca multibooked_eu; do
  echo $ds:$(python3 evaluate_single_dataset.py ../data/$ds/test.json ../baselines/graph_parser/experiments/$ds/head_final/test.conllu.json)
done;

opener_en:Sentiment Tuple F1: 0.525
opener_es:Sentiment Tuple F1: 0.502
darmstadt_unis:Sentiment Tuple F1: 0.211
multibooked_ca:Sentiment Tuple F1: 0.545
multibooked_eu:Sentiment Tuple F1: 0.573


# Baseline 2 - Sequence Labelling

This model extracts both expressions and relations using LSTM. Node Extractor in our work is inspired of this model's expression extractor.

In [ ]:
! cd semeval22_structured_sentiment/baselines/sequence_labeling && bash get_baselines.sh

In [ ]:
%%bash
cd semeval22_structured_sentiment/baselines/sequence_labeling
EMBEDDINGDIR="../graph_parser/embeddings"
for DATASET in darmstadt_unis mpqa multibooked_ca multibooked_eu norec opener_es opener_en; do
  python3 inference.py -data "$DATASET" -file test.json -device cuda
done;

Namespace(BATCH_SIZE=50, DATADIR='darmstadt_unis', DEVICE='cuda', FILE='test.json')
opening sources model...
opening targets model...
opening expressions model...
loading best relation prediction model...
Namespace(BATCH_SIZE=50, DATADIR='mpqa', DEVICE='cuda', FILE='test.json')
opening sources model...
opening targets model...
opening expressions model...
loading best relation prediction model...
Namespace(BATCH_SIZE=50, DATADIR='multibooked_ca', DEVICE='cuda', FILE='test.json')
opening sources model...
opening targets model...
opening expressions model...
loading best relation prediction model...
Namespace(BATCH_SIZE=50, DATADIR='multibooked_eu', DEVICE='cuda', FILE='test.json')
opening sources model...
opening targets model...
opening expressions model...
loading best relation prediction model...
Namespace(BATCH_SIZE=50, DATADIR='norec', DEVICE='cuda', FILE='test.json')
opening sources model...
opening targets model...
opening expressions model...
loading best relation prediction mod

tcmalloc: large alloc 2125438976 bytes == 0x14942000 @  0x7fc1b80b7001 0x7fc1b55411af 0x7fc1b5597c23 0x7fc1b5598a87 0x7fc1b563a823 0x5936cc 0x548c51 0x51566f 0x593dd7 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fc1b7cb2c87 0x5b621a
tcmalloc: large alloc 2125438976 bytes == 0x15d7e0000 @  0x7fc1b80b7001 0x7fc1b55411af 0x7fc1b5597c23 0x7fc1b5598a87 0x7fc1b563a823 0x5936cc 0x548c51 0x51566f 0x593dd7 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fc1b7cb2c87 0x5b621a
tcmalloc: large alloc 2125438976 bytes == 0x14942000 @  0x7fc1b80b7001 0x7fc1b55411af 0x7fc1b5597c23 0x7fc1b5598a87 0x7fc1b563a823 0x5936cc 0x548c51 0x51566f 0x593dd7 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fc1b7cb2c87 0x5b621a
tcmalloc: large alloc 2124947456 bytes == 0x15d7e0000 @  0x7fc1b80b7001 0x7fc1b55411af 0x7fc1b5597c23 0x7fc1b5598a87 0x7fc1b563a823 0x5936cc 0x548c51 0x5156

In [ ]:
%%bash
cd semeval22_structured_sentiment/evaluation
for ds in opener_en opener_es darmstadt_unis multibooked_ca multibooked_eu mpqa norec; do
  echo $ds:$(python3 evaluate_single_dataset.py ../data/$ds/test.json ../baselines/sequence_labeling/saved_models/relation_prediction/$ds/prediction.json)
done;

opener_en:Sentiment Tuple F1: 0.315
opener_es:Sentiment Tuple F1: 0.265
darmstadt_unis:Sentiment Tuple F1: 0.130
multibooked_ca:Sentiment Tuple F1: 0.337
multibooked_eu:Sentiment Tuple F1: 0.348
mpqa:Sentiment Tuple F1: 0.024
norec:Sentiment Tuple F1: 0.192
